In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import random


# Funciones

In [3]:
# Función para guardar los datos en un archivo JSON
def guardar_en_json(datos, nombre_archivo):
    try:
        with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
            # Escribir los datos en formato JSON, con indentación para mayor legibilidad
            json.dump(datos, archivo, ensure_ascii=False, indent=4)
    except Exception as e:
        print(f"Error al guardar el archivo JSON: {e}")



# Convertir los datos a un DataFrame de pandas
def convertir_a_dataframe(datos):
    df = pd.DataFrame(datos)
    return df


df_relojes = pd.DataFrame()

def parsear_relojes(links):
    global df_relojes  # Acceder al DataFrame global
    for link in links:
        url = f"https://www.chrono24.es{link}"
        browser = webdriver.Edge()

        try:
            # Cargar la página del reloj
            browser.get(url)
            time.sleep(random.randint(5, 10))  # Pausa para evitar bloqueos
            wait = WebDriverWait(browser, 10)

            # Si aparece un popup, cerrarlo
            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/dialog/div/div[2]/button"))).click()
            except Exception:
                pass  # Si no hay popup, continuar

            # Esperar a que la sección de detalles esté presente
            wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='tabpanel']")))

            # Obtener el HTML de la página
            html = browser.page_source
            soup = BeautifulSoup(html, "html.parser")

            # Buscar la tabla de características
            tabla = soup.find("div", {"role": "tabpanel"})
            if tabla:
                filas = tabla.find_all("tr")
                detalles = {}

                for fila in filas:
                    celdas = fila.find_all("td")
                    if len(celdas) >= 2:  # Procesar solo filas con pares clave-valor
                        clave = celdas[0].text.strip()
                        valor = celdas[1].text.strip()
                        # Limpiar espacios no rompibles
                        valor = valor.replace("\xa0", " ")
                        detalles[clave] = valor

                # Agregar los datos extraídos al DataFrame
                df_relojes = pd.concat([df_relojes, pd.DataFrame([detalles])], ignore_index=True)

                # Guardar los datos en un archivo JSON
                with open("datos_relojes.json", "w", encoding="utf-8") as archivo_json:
                    df_relojes.to_json(archivo_json, orient="records", force_ascii=False, indent=4)

                # Mostrar los datos extraídos
                print("\n Datos extraídos del reloj:")
                for clave, valor in detalles.items():
                    print(f"{clave}: {valor}")

            else:
                print("No se encontró el elemento 'tabpanel'.")

        except Exception as e:
            print(f"Error general al procesar el reloj {link}: {e}")

        finally:
            # Cerrar el navegador
            browser.quit()




# DATA DE UN RELOJ

In [14]:
# URL del reloj en Chrono24
url = "https://www.chrono24.es/rolex/rolex-datejust-41--id37077875.htm?searchHash=e7a6168c_looecP&pos=1"
browser = webdriver.Edge()

try:
    # Cargar la página
    browser.get(url)
    wait = WebDriverWait(browser, 10)

    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/dialog/div/div[2]/button"))).click()
    except Exception:
        pass  

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='tabpanel']")))

    # Obtener el HTML de la página
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    tabla = soup.find("div", {"role": "tabpanel"})
    if tabla:
        filas = tabla.find_all("tr")
        resultados = {}

        for fila in filas:
            celdas = fila.find_all("td")
            if len(celdas) >= 2:  # Solo procesar filas con pares clave-valor
                clave = celdas[0].text.strip()
                valor = celdas[1].text.strip()
                # Eliminar los espacios no rompibles (representados por '\xa0')
                valor = valor.replace("\xa0", " ")
                resultados[clave] = valor

        # Guardar los resultados en un archivo JSON
        guardar_en_json(resultados, 'datos_reloj.json')
        # Convertir los resultados en un DataFrame de pandas
        df = convertir_a_dataframe(resultados)

    else:
        print("No se encontró el elemento 'tabpanel'.")

except Exception as e:
    print("Error general:", e)

finally:
    browser.quit()

df


Error general: If using all scalar values, you must pass an index


,Código del anuncio,Marca,(Reloj) Modelo,Número de referencia,Calibre,Material de la caja,Material de la pulsera,Año de fabricación,Estado,Contenido de la entrega,...,Número de joyas,Diámetro,Resistente al agua,Material del bisel,Cristal,Esfera,Esfera con números,Color de la pulsera,Cierre,Material del cierre
0,M2PGZ3,Rolex,Datejust 41,126334,3235,Oro blanco,Acero,2022,Usado (Muy bueno)\n\n E...,"Con estuche original, con documentos originales",...,31,41 mm\n\nProbar,10 ATM,Oro blanco,Cristal de zafiro,Verde,Sin cifras,Acero,Cierre desplegable,Acero


# EXTRAER LINKS

In [4]:
links = []
marcas  = ["rolex", "audemarspiguet", "patekphilippe"]

for marca in marcas:
    # URL de toda la pagina
    url = f"https://www.chrono24.es/{marca}/index.htm"
    browser = webdriver.Edge()

    try:
        # Cargar la página
        browser.get(url)
        time.sleep(random.randint(10,15))
        wait = WebDriverWait(browser, 10)

        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/dialog/div/div[2]/button"))).click()
        except Exception:
            pass  

        wait.until(EC.presence_of_element_located((By.ID, "wt-watches")))

        # Obtener el html de la pagina
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")

        tabla = soup.find("div", {"id" : "wt-watches"})
        if tabla:
            anuncio = tabla.find_all("a")
            for reloj in anuncio:
                link = reloj.get("href")
                if link:
                    links.append(link)
        else:
            print("No se encontro el elemento 'wt-watches'")

    except Exception as e:
        print(f"Error general: {e}")

    finally:
        browser.quit()

### Links de las tres marcas

In [5]:
links

['/rolex/rolex-yacht-master--id35039453.htm',
 '/rolex/rolex-oyster-perpetual-date--id35646398.htm',
 '/rolex/rolex-submariner--id35973403.htm',
 '/rolex/rolex-gmt-master-ii-16710-pepsi-no-holes--id34829777.htm',
 '/rolex/rolex-daytona-cosmograph-steel--gold-y-serial-2002--id37078947.htm',
 '/rolex/lady-datejust--id34719536.htm',
 '/rolex/datejust-36-steel--gold-black-diamond-dial--id37754998.htm',
 '/rolex/rolex-like-new-condition-2022-gmt-master-ii--id37703813.htm',
 '/rolex/rolex-oyster-perpetual-date-automatic-steel--18k-gold-ladies-ref6917--id22329306.htm',
 '/rolex/rolex-submariner-date-patina-dial-ref-16610-from-1995--id36967126.htm',
 '/rolex/rolex-oyster-perpetual-41-like-new-99-2022-124300--id36288055.htm',
 '/rolex/datejust-36--id34119424.htm',
 '/rolex/rolex-lady-datejust--id35978010.htm',
 '/rolex/rolex-datejust-36--id37372974.htm',
 '/rolex/rolex-datejust-36-linen-dial--id35972519.htm',
 '/rolex/day-date-president--id37565200.htm',
 '/rolex/rolex-day-date-oysterquartz-18k

### Mezclar los links

In [16]:
shuffled_links = links.copy()
random.shuffle(shuffled_links)


shuffled_links = shuffled_links[0:50]
shuffled_links

['/audemarspiguet/audemars-piguet-royal-oak-offshore-chronograph---rose-gold---very-nice-condition---papers--id37127984.htm',
 '/patekphilippe/patek-philippe-nautilus--id37363381.htm',
 '/audemarspiguet/royal-oak-rose-gold---extract-incoming---top-conditions--id36789402.htm',
 '/rolex/rolex-new-service-2-year-warranty-oyster-perpetual-date-vintage--id37615307.htm',
 '/rolex/daytona--id34768417.htm',
 '/audemarspiguet/audemars-piguet-royal-oak-offshore-alinghi-polaris--ref26040st--unpolished--ap-extract---id36221361.htm',
 '/audemarspiguet/audemars-piguet-royal-oak-offshore-rubberclad-silver-green-arabic-lumes-full-set-box-papers-tag--id36554687.htm',
 '/rolex/rolex-oyster-perpetual-date-automatic-steel-ref16200--id36649144.htm',
 '/audemarspiguet/audemars-piguet-royal-oak-offshore-chronograph-rubber-clad-25940sk-watch-only--id36119637.htm',
 '/audemarspiguet/audemars-piguet-royal-oak-day-date-37mm-steel-gold--id35633521.htm',
 '/patekphilippe/patek-philippe-annual-calendar-ref5205g-010

In [17]:
parsear_relojes(shuffled_links)


 Datos extraídos del reloj:
Código del anuncio: M3S4W5
Marca: Audemars Piguet
(Reloj) Modelo: Royal Oak Offshore Chronograph
Número de referencia: 26400RO.OO.A002CA.01
Código del comerciante: K0030
Calibre: 3126/3840
Material de la caja: Oro rosa
Material de la pulsera: Caucho
Año de fabricación: 2011
Estado: Usado (Muy bueno)

                        El artículo presenta ligeras señales de uso, como unos leves arañazos, pero no son perceptibles.
Contenido de la entrega: Con documentos originales, sin estuche original
Género: Reloj de caballero/Unisex
Ubicación: Alemania, München
Precio: 34 690 €
Disponibilidad: Artículo en stock
Reserva de marcha: 50 h
Número de joyas: 59
Diámetro: 44 mm

Probar
Resistente al agua: 10 ATM
Material del bisel: Cerámica
Cristal: Cristal de zafiro
Esfera: Negro
Esfera con números: Sin cifras
Color de la pulsera: Negro
Cierre: Hebilla ardillón
Material del cierre: Oro rosa

 Datos extraídos del reloj:
Código del anuncio: M8TRP3
Marca: Patek Philippe
(Relo

In [2]:
ruta_json = "datos_relojes_chrono24.json"
df_relojes_chrono24 = pd.read_json(ruta_json)
df_relojes_chrono24

,Código del anuncio,Marca,(Reloj) Modelo,Número de referencia,Código del comerciante,Calibre,Material de la caja,Material de la pulsera,Año de fabricación,Estado,...,Cierre,Material del cierre,Última revisión,Calibre básico,Oscilación,Grosor,Longitud de la pulsera,Grosor de la pulsera,Ancho de la pulsera,Ancho de cierre
0,M3S4W5,Audemars Piguet,Royal Oak Offshore Chronograph,26400RO.OO.A002CA.01,K0030,3126/3840,Oro rosa,Caucho,2011,Usado (Muy bueno)\n\n E...,...,Hebilla ardillón,Oro rosa,None,None,None,None,None,None,None,None
1,M8TRP3,Patek Philippe,Nautilus,5712G-001\n(Nautilus Fase lunar),None,240 PS IRM C LU,Oro blanco,None,2009,Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Oro blanco,None,None,None,None,None,None,None,None
2,LWIVU7,Audemars Piguet,Royal Oak Selfwinding,15450OR.OO.D002CR.01\n(Royal Oak Selfwinding 3...,None,Automático,Oro rosa,Piel de aligátor,2018 (Año aproximado),Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Oro rosa,None,None,None,None,None,None,None,None
3,ME85N2,Rolex,Oyster Perpetual Date,1500\n(Oyster Perpetual Vintage),None,1570,Acero,Acero,1972,Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Acero,4 de diciembre de 2024\n\n ...,cal. 3035,None,None,None,None,None,None
4,KP7HD7,Rolex,Daytona,116506\n(Ice),None,4130,Platino,Platino,2022,Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Platino,None,4130,None,None,None,None,None,None
5,LKCKX8,Audemars Piguet,Royal Oak Offshore,26040ST.OO.D002CA.01,None,2326/2847,Acero,Caucho,2006,Usado (Bueno)\n\n El ar...,...,Cierre desplegable,Acero,None,None,None,None,None,None,None,None
6,LRHRZ2,Audemars Piguet,Royal Oak Offshore Chronograph,25940SK.OO.D002CA.02.A,None,2326/2840,Acero,Caucho,2008 (Año aproximado),Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Acero,None,Manufacture Rolex,28800 A/h,12 mm,170 mm (95 mm / 75 mm),4 mm,20 mm\n\nAsistente para determinar tamaño,16 mm
7,LTINS0,Rolex,None,16200,6706,Automático,Acero,Acero,1991,Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Acero,7 de octubre de 2024\n\n ...,None,None,123 mm,190 mm,310 mm,1840 mm\n\nAsistente para determinar tamaño,1520 mm
8,LI6390,Audemars Piguet,Royal Oak Offshore Chronograph,25940SK.OO.D002CA.01,None,2226/2840,Acero,Caucho,No conocido,Usado (Bueno)\n\n El ar...,...,Cierre desplegable,Acero,None,None,None,None,None,None,None,None
9,L7R010,Audemars Piguet,Royal Oak Day-Date,25594SA,None,AP 2224/2825,Acero y oro,Acero y oro,1987 (Año aproximado),Usado (Bueno)\n\n El ar...,...,Cierre desplegable,Acero,5 de julio de 2024\n\n ...,LeCoultre & Cie base,3 Hz,9 mm,172 mm,"2,5 mm",23 mm\n\nAsistente para determinar tamaño,15 mm


In [7]:
df = df_relojes_chrono24[(df_relojes_chrono24["Marca"] == "Audemars Piguet") & (df_relojes_chrono24["Número de referencia"] == "56175ST")]
df

,Código del anuncio,Marca,(Reloj) Modelo,Número de referencia,Código del comerciante,Calibre,Material de la caja,Material de la pulsera,Año de fabricación,Estado,...,Cierre,Material del cierre,Última revisión,Calibre básico,Oscilación,Grosor,Longitud de la pulsera,Grosor de la pulsera,Ancho de la pulsera,Ancho de cierre
41,LIWUW6,Audemars Piguet,Royal Oak,56175ST,None,2612,Acero,Acero,1991,Usado (Muy bueno)\n\n E...,...,Cierre desplegable,Acero,None,None,None,None,None,None,None,None
